In [1]:
%pip install optuna cebra==0.4.0 matplotlib==3.9.2 numpy pandas scipy seaborn umap_learn pyspark python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("/main/external/dimensionality-reduction")

In [3]:
import cebra
import torch
import torch.utils
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from utils import transform
import dotenv
import os
dotenv.load_dotenv()
dotenv.load_dotenv("/main/external/dimensionality-reduction/.env")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


MAX_MEMORY = "120g"

spark = SparkSession \
    .builder \
    .appName("UranusCluster") \
    .appName("Foo") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.memory.offHeap.enabled",True)\
    .config("spark.memory.offHeap.size","16g")   \
    .getOrCreate()

# Verify the SparkContext
print(spark.sparkContext.getConf().getAll())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/17 14:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.driver.memory', '120g'), ('spark.app.id', 'local-1734447249800'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'), ('spark.app.submitTime', '17344

# Data Load

In [5]:
#Taking just one because this still doesn`t support multi session data
DATA_PATH = "ID18150/Day6/DataFrame_Imaging_spiking_18150_day6"

In [6]:
df = spark.read.format("parquet").load("/main/external/data/transformed")\
    .select(["index","neural_data", "positional_encoding", "file_name"])\
    .where(F.col("file_name") == DATA_PATH)
df.show(5)

+------------+--------------------+--------------------+--------------------+
|       index|         neural_data| positional_encoding|           file_name|
+------------+--------------------+--------------------+--------------------+
|317827579904|[0.00353814595291...|0.014736815295795708|ID18150/Day6/Data...|
|317827707084|[7.32697662897408...|0.002792727523517...|ID18150/Day6/Data...|
|317827579905|[0.00376488942492...|0.014736815295795708|ID18150/Day6/Data...|
|317827707085|[5.27972821146249...|0.002792727523517...|ID18150/Day6/Data...|
|317827579906|[0.00368517413153...| 0.01473313965622418|ID18150/Day6/Data...|
+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [7]:
# n_neurons = max([row.neural_size for row in df.select(F.max(F.size("neural_data")).alias("neural_size")).collect()])
# n_rows = df.count()
indices = df.select('index').rdd.flatMap(lambda x: x).collect()
len(indices), max(indices), min(indices)

(131777, 317827711680, 317827579904)

In [8]:
def split(X, p):
    d = len(X) // p
    X_reduced = X[:(d * p)]
    partitions = np.split(np.array(X_reduced), p)
    partitions = [a.tolist() for a in partitions]
    if len(X[d*p:]) != 0:
        partitions += [X[d*p:]]
    return partitions

In [9]:
delta = 5000
def get_x_ticks(L:int):
    x_ticks = np.arange(0,L,delta)
    x_tick_labels = [f"{t[i]/100:.2f}" for i in x_ticks]
    return x_ticks, x_tick_labels

# CEBRA Encoder

# Models

In [ ]:
import logging

In [ ]:
partition_size = 512
kf = KFold(n_splits = 4)
latent_dimension = 8
scores = []

from cebra import CEBRA

multi_cebra_model = CEBRA(model_architecture='offset1-model',
                    batch_size=64,
                    learning_rate=3e-4,
                    temperature_mode='auto',
                    output_dimension=latent_dimension,
                    max_iterations=1000,
                    distance='cosine',
                    conditional='time_delta',
                    device='cuda_if_available',
                    verbose=True,
                    time_offsets=1)

In [12]:
decoder = torch.nn.Sequential(
    torch.nn.Linear(latent_dimension, 3),
    torch.nn.GELU(),
    torch.nn.Linear(3,3),
    torch.nn.GELU(),
    torch.nn.Linear(3,1),
    torch.nn.Tanh()
)

criterion = torch.nn.MSELoss()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=3e-4)


In [ ]:
scores_hist = []
for train_indices, test_indices in kf.split(indices):
    logging.info(f"Started with {len(train_indices)=} and {len(test_indices)=}")
    for train_partition_indices, test_partition_indices in zip(split(train_indices, partition_size),split(test_indices, partition_size)):
        logging.info("Starting new iteration")
        train_partition_indices = [[indices[i]] for i in train_partition_indices]
        test_partition_indices  = [[indices[i]] for i in test_partition_indices]
        
        logging.info("Filtering dataframe")
        df_train_index = spark.createDataFrame(train_partition_indices[:], ["index"])
        df_test_index  = spark.createDataFrame(test_partition_indices[:], ["index"])
        df_train = df_train_index.join(df, how = "left", on = "index")
        df_test  = df_test_index.join(df, how = "left", on = "index")
        
        logging.info("Grouping data")
        groups_train = [x[0] for x in df_train.select("file_name").distinct().collect()]
        groups_list_train = [df_train.filter(F.col("file_name")==x).collect() for x in groups_train]
        groups_test = [x[0] for x in df_test.select("file_name").distinct().collect()]
        groups_list_test = [df_test.filter(F.col("file_name")==x).collect() for x in groups_test]
        
        X_train = [torch.Tensor(list(map(lambda x: x.neural_data, group))) for group in groups_list_train]
        X_test = [torch.Tensor(list(map(lambda x: x.neural_data, group))) for group in groups_list_test]
        y_train = [torch.Tensor(list(map(lambda x: x.positional_encoding, group))) for group in groups_list_train]
        y_test = [torch.Tensor(list(map(lambda x: x.positional_encoding, group))) for group in groups_list_test]
        
        logging.info("Performing partial fits")
        for X,y in zip(X_train, y_train):
            with device:
                # Train Embedding
                multi_cebra_model.partial_fit(X, y) #Partial fit doesn`t work on multi session data :(
                embedding = torch.Tensor(transform(multi_cebra_model,X))
                
                # Train Decoder
                decoder.train()
                decoder_optimizer.zero_grad()
                predicted_embedding = decoder(embedding)
                loss = criterion(predicted_embedding, y)
                loss.backward()
                decoder_optimizer.step()
        
        scores = []
        for X,y in zip(X_test, y_test):
            with device:
                # Test
                test_embedding = torch.Tensor(transform(multi_cebra_model, X))
                decoder.eval()
                pred = decoder(test_embedding)
                score = r2_score(y.detach().numpy(), pred.detach().numpy())
                scores.append(score)
        mean_score = np.mean(scores)
        
        logging.info(f"Average score: {mean_score}")
        
        scores_hist.append(mean_score)
        
        

    

2024-12-17 15:03:10,685 - STDOUT - INFO - Starting new iteration


INFO:STDOUT:Starting new iteration


2024-12-17 15:03:10,686 - STDOUT - INFO - Filtering dataframe


INFO:STDOUT:Filtering dataframe


2024-12-17 15:03:10,708 - STDOUT - INFO - Grouping data


...

INFO:STDOUT:Filtering dataframe


2024-12-17 15:04:55,087 - STDOUT - INFO - Grouping data


INFO:STDOUT:Grouping data
ERROR:root:KeyboardInterrupt while sending command.===========>   (60 + 4) / 64]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
